# Avoiding Routes Using ORS Directions API

## Preprocessing

In [1]:
import folium

from openrouteservice import client

import fiona as fn
from shapely.ops import cascaded_union

import random
import pandas as pd
from datetime import datetime
from shapely.geometry import Polygon, mapping, MultiPolygon, LineString, Point, GeometryCollection
from shapely.ops import unary_union
from pyproj import Transformer
import json
from shapely import affinity
from decimal import Decimal, getcontext

In [2]:
# insert your ORS api key
api_key = 'API_KEY'
ors = client.Client(key=api_key)

## Data Inspection
Outputs from running scripts against the pickle file. These are inspected to determine the best threshold to set that distinguishes an area as busy, for each busy area a polygon is created.

### Noise Crime Model

In [4]:
df = pd.read_csv("../backend/quietquestapp/quietquestapp_noiselocations.csv", names=['index', 'long','lat','hour','weekday','weekend',"count"], skiprows=1)
df = df[['long','lat','hour','weekday','weekend',"count"]]

columns = df.columns

for column in columns:
    print("Column Name:", column, "\nData Type:", df[column].dtype, "\nNumber of Unique Values:", str(len(df[column].unique())))
    print(df[column].unique())
    print()

# df_count = df[["count"]]
# print(df_count.value_counts())

# hour_values_list = df.loc[df["count"] >= 0.5, "hour", "weekday"].value_counts()
# print(hour_values_list)

# df_weekday = df[["weekday"]].value_counts()
# print(df_weekday)

# df_weekend = df[["weekend"]].value_counts()
# print(df_weekend)

# df_hour_weekday = df[["hour", "weekday"]].value_counts()
# df_hour_weekday

df_hour_weekday1 = df.loc[df["count"] <= 0.19, ["hour", "weekday"]].value_counts()
print(df_hour_weekday1)

df_hour_weekend1 = df.loc[df["count"] <= 0.19, ["hour", "weekday"]].value_counts()
print(df_hour_weekend1)

df_hour_weekday2 = df.loc[(0.20 <= df["count"]) & (df["count"] <= 0.39), ["hour", "weekday"]].value_counts()
print(df_hour_weekday2)

df_hour_weekend2 = df.loc[(0.20 <= df["count"]) & (df["count"] <= 0.39), ["hour", "weekday"]].value_counts()
print(df_hour_weekend2)

df_hour_weekday3 = df.loc[(0.40 <= df["count"]) & (df["count"] <= 0.59), ["hour", "weekday"]].value_counts()
print(df_hour_weekday3)

df_hour_weekend3 = df.loc[(0.40 <= df["count"]) & (df["count"] <= 0.59), ["hour", "weekday"]].value_counts()
print(df_hour_weekend3)

df_hour_weekday4 = df.loc[(0.60 <= df["count"]) & (df["count"] <= 0.79), ["hour", "weekday"]].value_counts()
print(df_hour_weekday4)

df_hour_weekend4 = df.loc[(0.60 <= df["count"]) & (df["count"] <= 0.79), ["hour", "weekday"]].value_counts()
print(df_hour_weekend4)

df_hour_weekday5 = df.loc[(0.80 <= df["count"]) & (df["count"] <= 1), ["hour", "weekday"]].value_counts()
print(df_hour_weekday5)

df_hour_weekend5 = df.loc[(0.80 <= df["count"]) & (df["count"] <= 1), ["hour", "weekday"]].value_counts()
print(df_hour_weekend5)

Column Name: long 
Data Type: float64 
Number of Unique Values: 19484
[-73.92359    -73.97609401 -73.96705337 ... -73.95677104 -74.00344589
 -74.00153326]

Column Name: lat 
Data Type: float64 
Number of Unique Values: 19530
[40.86685084 40.77717631 40.76910729 ... 40.76986144 40.73353454
 40.72242106]

Column Name: hour 
Data Type: int64 
Number of Unique Values: 24
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]

Column Name: weekday 
Data Type: int64 
Number of Unique Values: 2
[1 0]

Column Name: weekend 
Data Type: int64 
Number of Unique Values: 2
[0 1]

Column Name: count 
Data Type: float64 
Number of Unique Values: 1628
[0.2464 0.1328 0.2728 ... 0.9216 0.884  0.9128]

hour  weekday
6     0          73519
5     0          73501
7     0          73333
11    0          73270
8     0          72968
9     0          72832
10    0          72647
21    0          72635
3     0          72441
14    0          71767
4     0          71582
13    0          7151

### Taxi Weekday Crime Model

In [3]:
df = pd.read_csv("../backend/quietquestapp/quietquestapp_taxiweekdaylocations.csv", names=["index","long","lat","hour","day","count"], skiprows=1)
df = df[['long','lat', 'hour','day',"count"]]

columns = df.columns

# for column in columns:
#     print("Column Name:", column, "\nData Type:", df[column].dtype, "\nNumber of Unique Values:", str(len(df[column].unique())))
#     print(df[column].unique())
#     print()

# df_count = df[["count"]]
# print(df_count.value_counts())

df_hour_day1 = df.loc[df["count"] <= 0.19, ["hour", "day"]].value_counts()
df_hour_day1

hour  day
4     1      867974
3     1      867966
4     3      867464
      2      866515
2     0      864847
              ...  
18    4       54698
      3       54302
19    2       50797
      3       47897
      4       46378
Name: count, Length: 120, dtype: int64

In [4]:
df_hour_day2 = df.loc[(0.20 <= df["count"]) & (df["count"] <= 0.39), ["hour", "day"]].value_counts()
df_hour_day2

hour  day
16    3      323729
      4      302330
      2      301806
      1      288116
11    0      284777
              ...  
2     0        2679
      1        2128
4     2        1244
      4        1109
      3         365
Name: count, Length: 118, dtype: int64

In [5]:
df_hour_day3 = df.loc[(0.40 <= df["count"]) & (df["count"] <= 0.59), ["hour", "day"]].value_counts()
df_hour_day3

hour  day
15    4      243791
17    1      241933
13    2      240664
17    3      237986
13    4      235563
              ...  
1     2        2136
4     4        1827
1     0        1454
3     3         353
2     3         275
Name: count, Length: 110, dtype: int64

In [6]:
df_hour_day4 = df.loc[(0.60 <= df["count"]) & (df["count"] <= 0.79), ["hour", "day"]].value_counts()
df_hour_day4

hour  day
18    4      275393
      3      270166
19    3      266796
20    4      255649
18    1      255566
              ...  
5     0        2553
1     3        2167
3     3        1123
2     3        1122
5     1         421
Name: count, Length: 104, dtype: int64

In [7]:
df_hour_day5 = df.loc[(0.80 <= df["count"]) & (df["count"] <= 1), ["hour", "day"]].value_counts()
df_hour_day5

hour  day
19    4      285807
      3      268236
      2      267611
18    1      258105
      2      252816
              ...  
1     4        6646
5     1        4828
      2        4381
      3        4381
0     3        2014
Name: count, Length: 98, dtype: int64

In [8]:
df_hour_day1 = df.loc[df["count"] <= 0.19, ["hour"]].value_counts()
df_hour_day1

hour
4       4331231
3       4298014
2       4266376
5       4183748
1       4112875
0       3398258
6       3306380
23      1645759
7       1612272
16      1201207
10      1145153
11      1020454
9        976633
13       949370
8        937104
12       926157
15       858302
14       799380
22       739367
17       585903
21       497792
20       434990
19       312350
18       305138
Name: count, dtype: int64

In [9]:
df_hour_day2 = df.loc[(0.20 <= df["count"]) & (df["count"] <= 0.39), ["hour"]].value_counts()
df_hour_day2

hour
16      1498692
11      1265940
15      1232838
10      1198946
13      1161514
14      1156936
12      1138478
17      1136956
22      1052345
7       1044875
23      1042258
9       1026437
8        988667
21       889498
20       832452
19       617576
0        580275
18       541450
6        457428
1        159976
5         78608
2         42110
3         23198
4          6086
Name: count, dtype: int64

In [10]:
df_hour_day3 = df.loc[(0.40 <= df["count"]) & (df["count"] <= 0.59), ["hour"]].value_counts()
df_hour_day3

hour
17      1161845
15      1123970
13      1076004
14      1033176
12       997192
21       942166
11       934076
20       929072
18       924903
22       919166
16       905453
19       870489
10       852625
9        851980
8        841317
23       704606
7        651610
0        225086
6        179668
1         34038
5         30622
2         13434
3          8347
4          1827
Name: count, dtype: int64

In [11]:
df_hour_day4 = df.loc[(0.60 <= df["count"]) & (df["count"] <= 0.79), ["hour"]].value_counts()
df_hour_day4

hour
18      1291397
19      1223090
20      1129505
21      1052843
17       898754
14       830593
22       822915
12       758925
9        724401
15       698375
8        667887
10       656972
13       650725
11       617453
23       574408
16       433423
7        401841
6        148361
0         83088
5         29070
1         15290
3          6874
2          4874
Name: count, dtype: int64

In [12]:
df_hour_day5 = df.loc[(0.80 <= df["count"]) & (df["count"] <= 1), ["hour"]].value_counts()
df_hour_day5

hour
19      1237622
18      1207123
20       938051
21       877483
8        832513
22       729506
9        681606
7        563853
17       481873
14       429244
12       423057
13       408695
11       408651
10       399499
15       342968
23       308293
16       215518
6        210586
0         27106
5         13590
2          8425
1          6646
Name: count, dtype: int64

### Taxi Weekend Crime Model

In [4]:
df = pd.read_csv("../backend/quietquestapp/quietquestapp_taxiweekendlocations.csv", names=["index","long","lat","hour","day","count"], skiprows=1)
df = df[['long','lat', 'hour','day',"count"]]

columns = df.columns

# for column in columns:
#     print("Column Name:", column, "\nData Type:", df[column].dtype, "\nNumber of Unique Values:", str(len(df[column].unique())))
#     print(df[column].unique())
#     print()

# df_count = df[["count"]]
# print(df_count.value_counts())

df_hour_day = df.loc[df["count"] >= 0.8, ["hour", "day"]].value_counts()
df_hour_day

hour  day
0     6      34803
1     6      34577
0     5      33127
1     5      32914
2     6      32151
      5      32124
3     6      17387
      5      17257
13    6       9500
15    6       9500
17    6       9500
14    6       9500
11    6       9500
12    6       9500
16    6       9500
18    6       9033
19    6       9033
20    6       9033
21    6       9033
17    5       1400
14    5       1400
16    5       1400
15    5       1400
13    5       1400
12    5       1400
11    5       1400
18    5       1390
19    5        904
10    5        904
23    5         23
Name: count, dtype: int64

## Data Models
Setup reading from the outputted csvs for visualisation

### Noise Crime Model

In [3]:
def predicted_locations_noise_export(hour, day, count):
    df = pd.read_csv("../backend/quietquestapp/quietquestapp_noiselocations.csv", names=['index', 'long','lat','hour','weekday','weekend',"count"], skiprows=1)
    df = df[['long','lat','hour','weekday','weekend',"count"]]

    # Filter locations based on hour and day
    if 0 <= day <= 4:
        locations = df[(df["hour"] == int(hour)) & (df["weekday"] == 1) & (df["count"] >= count)]
    else:
        locations = df[(df["hour"] == int(hour)) & (df["weekend"] == 1) & (df["count"] >= count)]

    response_list = []
    for _, location in locations.iterrows():
        response_dict = {
            'long': location["long"],
            'lat': location["lat"],
        }
        response_list.append(response_dict)
    
    return response_list

### Taxi Weekday Crime Model

In [4]:
def predicted_locations_weekday_export(hour, day, count):
    df = pd.read_csv("../backend/quietquestapp/quietquestapp_taxiweekdaylocations.csv", names=["index","long","lat","hour","day","count"], skiprows=1)
    df = df[['long','lat', 'hour','day',"count"]]

    # Filter locations based on hour and day
    locations = df[(df["hour"] == int(hour)) & (df["day"] == int(day)) & (df["count"] >= count)]

    response_list = []
    for _, location in locations.iterrows():
        response_dict = {
            'long': location["long"],
            'lat': location["lat"],
        }
        response_list.append(response_dict)
    
    return response_list

### Taxi Weekend Crime Model

In [5]:
def predicted_locations_weekend_export(hour, day, count):
    df = pd.read_csv("../backend/quietquestapp/quietquestapp_taxiweekdaylocations.csv", names=["index","long","lat","hour","day","count"], skiprows=1)
    df = df[['long','lat', 'hour','day',"count"]]

    # Filter locations based on hour and day
    locations = df[(df["hour"] == int(hour)) & (df["day"] == int(day)) & (df["count"] >= count)]

    response_list = []
    for _, location in locations.iterrows():
        response_dict = {
            'long': location["long"],
            'lat': location["lat"],
        }
        response_list.append(response_dict)
    
    return response_list

## Setup Functions

In [7]:
# Function to create buffer around tweet point geometries and transform it to the needed coordinate system (WGS84)
def create_buffer_polygon(point_in, resolution=10, radius=100):
    transformer_wgs84_to_utm32n = Transformer.from_crs("EPSG:4326", "EPSG:3857")
    transformer_utm32n_to_wgs84 = Transformer.from_crs("EPSG:3857", "EPSG:4326")
    point_in_proj = transformer_wgs84_to_utm32n.transform(*point_in)
    point_buffer_proj = Point(point_in_proj).buffer(radius, resolution=resolution)
    
    # Adjust the aspect ratio of the buffer
    transformed_buffer = affinity.scale(point_buffer_proj, xfact=1, yfact=5)
    
    # Transform back to WGS84
    poly_wgs = [transformer_utm32n_to_wgs84.transform(*point) for point in transformed_buffer.exterior.coords]
    return poly_wgs


# Function to request directions with avoided_polygon feature
def create_route(data, avoided_point_list, n=0):
    print(mapping(MultiPolygon(avoided_point_list)))
    route_request = {'coordinates': data,
                        'format': 'geojson',
                        'profile': 'foot-walking',
                        'preference': 'shortest',
                        'instructions': True,
                        'options': {'avoid_polygons': mapping(MultiPolygon(avoided_point_list))}}
    route_directions = ors.directions(**route_request)
    return route_directions


# Function to create buffer around requested route
def create_buffer(r_directions):
    coordinates = r_directions['features'][0]['geometry']['coordinates']
    linestring = LineString(coordinates)
    dilated_route = linestring.buffer(0.0005)
    return dilated_route

def style_function(color):  # To style data
    return lambda feature: dict(color=color)

# Function to merge intersecting polygons efficiently
def merge_intersecting_polygons(poly_list):
    merged_polygons = []
    for poly in poly_list:
        if not merged_polygons:
            merged_polygons.append(poly)
        else:
            new_merged = []
            for merged_poly in merged_polygons:
                if poly.intersects(merged_poly):
                    poly = poly.union(merged_poly)
                else:
                    new_merged.append(merged_poly)
            new_merged.append(poly)
            merged_polygons = new_merged
    return merged_polygons

## Merged Routing
Visualises the routing that takes place in the backend for a particular day and time

In [14]:
# Create map
map = folium.Map(tiles='Stamen Toner', location=[40.76657321777155, -73.9831392189498], zoom_start=13)

#coordinates = [[-73.981386, 40.767868], [-73.973350, 40.764504]]

#coordinates = [[-73.992597, 40.763375], [-73.987212, 40.761124]]

#coordinates = [[-73.948918, 40.812560], [-73.944112, 40.810497]]

#coordinates = [[-73.94086360931398, 40.81892462451646], [-73.93900752067567, 40.82151464975118]]

#coordinates = [[-73.99379929637948, 40.7283915], [-73.9942, 40.7483]]

coordinates = [[-73.953416, 40.822715], [-73.956237, 40.818875]]

folium.map.Marker(
    location=list(reversed(coordinates[0])),
    icon=folium.Icon(color="red", icon="A"),
    ).add_to(map)

folium.map.Marker(
    location=list(reversed(coordinates[1])),
    icon=folium.Icon(color="blue", icon="B"),
    ).add_to(map)

# for coord in coordinates:
#     folium.map.Marker(list(reversed(coord))).add_to(map)

# Initializations
now = datetime.now()
prediction_hour = "1"
prediction_day = 6

high_index_value_ls = []
point_geometry = []

# Get predicted locations
all_locations = predicted_locations_noise_export(prediction_hour, prediction_day, 0.8)

for location in all_locations:
    position = [location['long'], location['lat']]
    point_buffer = Polygon(create_buffer_polygon(position))
    high_index_value_ls.append(point_buffer)
    point_geometry.append(Polygon(point_buffer))
    #folium.map.Marker(list(reversed(position)), 
    #                  icon=folium.Icon(color='lightgray', icon_color='blue',)
    #                  ,popup='High Count Value').add_to(map)

start_buffer = Polygon(create_buffer_polygon(coordinates[0], 10, 50))
end_buffer = Polygon(create_buffer_polygon(coordinates[1], 10, 50))

union_poly = unary_union(point_geometry)
union_poly_geojson = mapping(union_poly)

# Convert to GeoJSON-compatible format
union_poly_json = json.dumps(union_poly_geojson)
union_poly_data = json.loads(union_poly_json)

folium.features.GeoJson(data=union_poly_data,
                        name='Busy Areas',
                        style_function=style_function('#ffd699')).add_to(map)

avoided_point_list = []

# Create regular route with still empty avoided_point_list
optimal_directions = create_route(coordinates, avoided_point_list)

# Create buffer around route
avoidance_directions = create_buffer(optimal_directions)

folium.features.GeoJson(
                    data=optimal_directions,
                    name='Regular Route',
                    style_function=style_function('#ff5050'), overlay=True).add_to(map)

print('Generated regular route.')

# Initialise avoidance_route as an empty string
avoidance_route = ""

# Merge intersecting polygons
intersecting_polygons = merge_intersecting_polygons(high_index_value_ls)
avoidance_route = ""
avoided_points = []

intersecting_polygons_copy = intersecting_polygons.copy()

for poly in intersecting_polygons_copy:
    if poly.intersects(start_buffer) or poly.intersects(end_buffer):
        intersecting_polygons.remove(poly)

try:
    print("Number Unmerged Polygons: " + str(len(high_index_value_ls)))
    print("Total Merged Polygons: " + str(len(intersecting_polygons)))
    avoidance_route = create_route(coordinates, intersecting_polygons, 1)
    avoidance_directions = create_buffer(avoidance_route)

    print('Generated alternative route, which avoids affected areas.')
except Exception as e:
    print(e)
    
folium.features.GeoJson(
                data=avoidance_route,
                name='Alternative Route',
                style_function=style_function('#006600'),
                overlay=True).add_to(map)

folium.features.GeoJson(data=avoidance_directions,
                        name='Avoidance Directions Buffer',
                        style_function=style_function('#660061'),
                        overlay=True).add_to(map)

map.add_child(folium.map.LayerControl())
map

{'type': 'MultiPolygon', 'coordinates': []}
Generated regular route.
Number Unmerged Polygons: 148
Total Merged Polygons: 1
{'type': 'MultiPolygon', 'coordinates': [(((-73.9549847956875, 40.82196483778726), (-73.9548048190816, 40.821910893961544), (-73.95475330302698, 40.82188805066798), (-73.95474702545243, 40.82188616911677), (-73.95473081951155, 40.82187898308963), (-73.95471812773174, 40.82187517904214), (-73.95469034330658, 40.821862858913086), (-73.95468561873025, 40.82186144284083), (-73.95463410083198, 40.821838598895), (-73.95462782518835, 40.821836717936144), (-73.95457630639466, 40.821813873673385), (-73.95457003168941, 40.821811993002385), (-73.95440392877727, 40.82173834055462), (-73.95425579260584, 40.8216467930525), (-73.9541292713086, 40.821539604699375), (-73.95402748078918, 40.82141941482822), (-73.9539529279456, 40.82128918291256), (-73.95390744889525, 40.82115211569437), (-73.95389216372669, 40.821011588223456), (-73.95390744889525, 40.820871060752545), (-73.9539529

## Mapping

Creates a visualisation of the busy areas on the map for a given day and time

In [7]:
# Create map
map = folium.Map(tiles='Stamen Toner', location=[40.76657321777155, -73.9831392189498], zoom_start=13)

# Initializations
now = datetime.now()
prediction_hour = "19"
prediction_day = 4

high_index_value_ls = []
point_geometry = []

# Get predicted locations
all_locations = predicted_locations_weekday_export(prediction_hour, prediction_day, 0.8)
print("Number of Locations: " + str(len(all_locations)))

for location in all_locations:
    position = [location['long'], location['lat']]
    point_buffer = create_buffer_polygon(position)
    high_index_value_ls.append(point_buffer)
    point_geometry.append(Polygon(point_buffer))
    #folium.map.Marker(list(reversed(position)), 
    #                  icon=folium.Icon(color='lightgray', icon_color='blue',)
    #                  ,popup='High Count Value').add_to(map)

union_poly = unary_union(point_geometry)
union_poly_geojson = mapping(union_poly)

# Convert to GeoJSON-compatible format
union_poly_json = json.dumps(union_poly_geojson)
union_poly_data = json.loads(union_poly_json)

folium.features.GeoJson(data=union_poly_data,
                        name='Busy Areas',
                        style_function=style_function('#ffd699')).add_to(map)

map.add_child(folium.map.LayerControl())
map

KeyboardInterrupt: 

Exports an image of a map for every day and time possibility for inspection. Helps to determine the correct threshold that defines an area as busy

### Noise Model Mapping

In [17]:
# Create a loop for each day of the week (0-6)
for prediction_day in [0, 6]:
    # Create a map for every hour in a day (0-24)
    for prediction_hour in range(24):
        # Create map
        map = folium.Map(tiles='Stamen Toner', location=[40.76657321777155, -73.9831392189498], zoom_start=12)

        high_index_value_ls = []
        point_geometry = []

        # # Set the desired precision (number of decimal places)
        # getcontext().prec = 1

        # # Get predicted locations for the specific hour and day
        # count = 0.2
        # all_locations = predicted_locations_noise_export(str(prediction_hour), prediction_day, count)

        # while len(all_locations) <= 200 or len(all_locations) >= 3000:
        #     count += 0.1
        #     count = int(count)
        #     if count <= 0.9:
        #         count = 0.9  # Set count to 0 if it becomes zero or negative
        #         all_locations = predicted_locations_noise_export(str(prediction_hour), prediction_day, count)
        #         break  # Exit the loop if count becomes zero
        #     all_locations = predicted_locations_noise_export(str(prediction_hour), prediction_day, count)
        
        all_locations = predicted_locations_noise_export(str(prediction_hour), prediction_day, 0.5)

        print(str(prediction_day), str(prediction_hour))
        print("Number of Locations: " + str(len(all_locations)))
        
        # for location in all_locations:
        #     position = [location['long'], location['lat']]
        #     point_buffer = create_buffer_polygon(position)
        #     high_index_value_ls.append(point_buffer)
        #     point_geometry.append(Polygon(point_buffer))

        # union_poly = unary_union(point_geometry)
        # union_poly_geojson = mapping(union_poly)

        # # Convert to GeoJSON-compatible format
        # union_poly_json = json.dumps(union_poly_geojson)
        # union_poly_data = json.loads(union_poly_json)

        # folium.features.GeoJson(data=union_poly_data,
        #                         name='Busy Areas',
        #                         style_function=style_function('#ffd699')).add_to(map)

        # # Save the map as an image
        # map.save(f'map_noise_{prediction_day}_{prediction_hour}.html')


0 0
Number of Locations: 1297
0 1
Number of Locations: 227
0 2
Number of Locations: 35
0 3
Number of Locations: 0
0 4
Number of Locations: 0
0 5
Number of Locations: 0
0 6
Number of Locations: 0
0 7
Number of Locations: 0
0 8
Number of Locations: 0
0 9
Number of Locations: 0
0 10
Number of Locations: 0
0 11
Number of Locations: 0
0 12
Number of Locations: 0
0 13
Number of Locations: 0
0 14
Number of Locations: 0
0 15
Number of Locations: 0
0 16
Number of Locations: 0
0 17
Number of Locations: 0
0 18
Number of Locations: 118
0 19
Number of Locations: 211
0 20
Number of Locations: 627
0 21
Number of Locations: 3872
0 22
Number of Locations: 8511
0 23
Number of Locations: 10790
6 0
Number of Locations: 585
6 1
Number of Locations: 250
6 2
Number of Locations: 228
6 3
Number of Locations: 0
6 4
Number of Locations: 0
6 5
Number of Locations: 0
6 6
Number of Locations: 0
6 7
Number of Locations: 0
6 8
Number of Locations: 0
6 9
Number of Locations: 0
6 10
Number of Locations: 0
6 11
Number 

### Taxi Weekday Mapping

In [19]:
# Create a loop for each day of the week (0-6)
for prediction_day in range(5):
    # Create a map for every hour in a day (0-24)
    for prediction_hour in range(24):
        # Create map
        # map = folium.Map(tiles='Stamen Toner', location=[40.76657321777155, -73.9831392189498], zoom_start=12)

        # high_index_value_ls = []
        # point_geometry = []

        # # Set the desired precision (number of decimal places)
        # getcontext().prec = 1

        # # Get predicted locations for the specific hour and day
        # count = Decimal(0.2)
        # all_locations = predicted_locations_weekday_export(str(prediction_hour), prediction_day, count)

        # while len(all_locations) <= 30000 or len(all_locations) >= 60000:
        #     count += Decimal(0.1)
        #     if count <= 0.9:
        #         count = 0.9  # Set count to 0 if it becomes zero or negative
        #         all_locations = predicted_locations_weekday(str(prediction_hour), prediction_day, count)
        #         break  # Exit the loop if count becomes zero

        #     all_locations = predicted_locations_weekday(str(prediction_hour), prediction_day, count)

        all_locations = predicted_locations_weekday_export(str(prediction_hour), prediction_day, 0.8)

        print(str(prediction_day), str(prediction_hour))
        print("Number of Locations: " + str(len(all_locations)))

        # for location in all_locations:
        #     position = [location['long'], location['lat']]
        #     point_buffer = create_buffer_polygon(position)
        #     high_index_value_ls.append(point_buffer)
        #     point_geometry.append(Polygon(point_buffer))

        # union_poly = unary_union(point_geometry)
        # union_poly_geojson = mapping(union_poly)

        # # Convert to GeoJSON-compatible format
        # union_poly_json = json.dumps(union_poly_geojson)
        # union_poly_data = json.loads(union_poly_json)

        # folium.features.GeoJson(data=union_poly_data,
        #                         name='Busy Areas',
        #                         style_function=style_function('#ffd699')).add_to(map)

        # # Save the map as an image
        # map.save(f'map_weekday_{prediction_day}_{prediction_hour}.html')


KeyboardInterrupt: 

## Taxi Weekend Mapping

In [ ]:
# Create a loop for each day of the week (0-6)
for prediction_day in range(5, 7):
    # Create a map for every hour in a day (0-24)
    for prediction_hour in range(24):
        # Create map
        # map = folium.Map(tiles='Stamen Toner', location=[40.76657321777155, -73.9831392189498], zoom_start=12)

        # high_index_value_ls = []
        # point_geometry = []

        # # Set the desired precision (number of decimal places)
        # getcontext().prec = 1

        # # Get predicted locations for the specific hour and day
        # count = Decimal('1.0')
        # all_locations = predicted_locations_weekend_export(str(prediction_hour), prediction_day, count)

        # while len(all_locations) < 800 or len(all_locations) > 2200:
        #     count -= Decimal(0.1)
        #     if count <= 0.1:
        #         count = 0.1  # Set count to 0 if it becomes zero or negative
        #         all_locations = predicted_locations_weekend(str(prediction_hour), prediction_day, count)
        #         break  # Exit the loop if count becomes zero

        #     all_locations = predicted_locations_noise(str(prediction_hour), prediction_day, count)
        
        all_locations = predicted_locations_weekend_export(str(prediction_hour), prediction_day, 0.8)

        print(str(prediction_day), str(prediction_hour))
        print("Number of Locations: " + str(len(all_locations)))

        # for location in all_locations:
        #     position = [location['long'], location['lat']]
        #     point_buffer = create_buffer_polygon(position)
        #     high_index_value_ls.append(point_buffer)
        #     point_geometry.append(Polygon(point_buffer))

        # union_poly = unary_union(point_geometry)
        # union_poly_geojson = mapping(union_poly)

        # # Convert to GeoJSON-compatible format
        # union_poly_json = json.dumps(union_poly_geojson)
        # union_poly_data = json.loads(union_poly_json)

        # folium.features.GeoJson(data=union_poly_data,
        #                         name='Busy Areas',
        #                         style_function=style_function('#ffd699')).add_to(map)

        # # Save the map as an image
        # map.save(f'map_weekend_{prediction_day}_{prediction_hour}.html')


Number of Busy Places: 3728
Number of Busy Places: 3185
Number of Busy Places: 2788
Number of Busy Places: 1656
Number of Busy Places: 212
Number of Busy Places: 0
Number of Busy Places: 6
Number of Busy Places: 26
Number of Busy Places: 148
Number of Busy Places: 609
Number of Busy Places: 1532
Number of Busy Places: 2328
Number of Busy Places: 3268
Number of Busy Places: 3390
Number of Busy Places: 2583
Number of Busy Places: 3001
Number of Busy Places: 2111
Number of Busy Places: 3370
Number of Busy Places: 5612
Number of Busy Places: 5533
Number of Busy Places: 3358
Number of Busy Places: 3690
Number of Busy Places: 4702
Number of Busy Places: 5277
Number of Busy Places: 4488
Number of Busy Places: 3959
Number of Busy Places: 2870
Number of Busy Places: 1965
Number of Busy Places: 228
Number of Busy Places: 0
Number of Busy Places: 0
Number of Busy Places: 0
Number of Busy Places: 67
Number of Busy Places: 191
Number of Busy Places: 738
Number of Busy Places: 1690
Number of Busy Pl

## Number Merged Polygons

Displays the number of total polygons created for all coordinates and the resultant number of polygons after all merging has taken place

In [73]:
# Initializations
now = datetime.now()
prediction_hour = "22"
prediction_day = 3

high_index_value_ls = []

# Get predicted locations
all_locations = predicted_locations_noise_export(prediction_hour, prediction_day)

for location in all_locations:
    position = [location['long'], location['lat']]
    point_buffer = Polygon(create_buffer_polygon((location.long, location.lat)))
    high_index_value_ls.append(point_buffer)

intersecting_polygons = merge_intersecting_polygons(high_index_value_ls)

print("Number Unmerged Polygons to API: "+ str(len(all_locations)))
print("Number Merged Polygons to API: "+ str(len(intersecting_polygons)))

Number of Busy Places: 3313
Number Unmerged Polygons to API: 3313
Number Merged Polygons to API: 40


## Multiple Attempts To Get Route Method

The original method was to iteratively discover each busy area and send a new API call with the newly included polygon. However, after merging it is more efficient to send all the polygons at once to the API service as there are far fewer due to clustering of coordinates observed empircially.

In [ ]:
# Create map
map = folium.Map(tiles='Stamen Toner', location=[40.76657321777155, -73.9831392189498], zoom_start=13)

#coordinates = [[-73.981386, 40.767868], [-73.973350, 40.764504]]

#coordinates = [[-73.992597, 40.763375], [-73.987212, 40.761124]]

#coordinates = [[-73.948918, 40.812560], [-73.944112, 40.810497]]

#coordinates = [[-73.94086360931398, 40.81892462451646], [-73.93900752067567, 40.82151464975118]]

coordinates = [[-73.99379929637948, 40.7283915], [-73.9942, 40.7483]]

for coord in coordinates:
    folium.map.Marker(list(reversed(coord))).add_to(map)

# Initializations
now = datetime.now()
prediction_hour = "22"
prediction_day = 3

high_index_value_ls = []
point_geometry = []

# Get predicted locations
all_locations = predicted_locations(prediction_hour, prediction_day)

for location in all_locations:
    position = [location['long'], location['lat']]
    point_buffer = create_buffer_polygon(position)
    high_index_value_ls.append(point_buffer)
    point_geometry.append(Polygon(point_buffer))
    #folium.map.Marker(list(reversed(position)), 
    #                  icon=folium.Icon(color='lightgray', icon_color='blue',)
    #                  ,popup='High Count Value').add_to(map)

union_poly = unary_union(point_geometry)
union_poly_geojson = mapping(union_poly)

# Convert to GeoJSON-compatible format
union_poly_json = json.dumps(union_poly_geojson)
union_poly_data = json.loads(union_poly_json)

folium.features.GeoJson(data=union_poly_data,
                        name='Busy Areas',
                        style_function=style_function('#ffd699')).add_to(map)

avoided_point_list = []

# Create regular route with still empty avoided_point_list
optimal_directions = create_route(coordinates, avoided_point_list)

# Create buffer around route
avoidance_directions = create_buffer(optimal_directions)

folium.features.GeoJson(
                    data=optimal_directions,
                    name='Regular Route',
                    style_function=style_function('#ff5050'), overlay=True).add_to(map)

print('Generated regular route.')

# Initialise avoidance_route as an empty string
avoidance_route = ""
status = "no_rerouting"  # Indicates no rerouting was needed
attempts = 0

all_polygons = []
for site_poly in high_index_value_ls:
    poly = Polygon(site_poly)
    all_polygons.append(poly)

# Function to merge intersecting polygons efficiently
def merge_intersecting_polygons(poly_list):
    merged_polygons = []
    for poly in poly_list:
        if not merged_polygons:
            merged_polygons.append(poly)
        else:
            new_merged = []
            for merged_poly in merged_polygons:
                if poly.intersects(merged_poly):
                    poly = poly.union(merged_poly)
                else:
                    new_merged.append(merged_poly)
            new_merged.append(poly)
            merged_polygons = new_merged
    return merged_polygons

# Merge intersecting polygons
intersecting_polygons = merge_intersecting_polygons(all_polygons)
avoidance_route = ""
avoided_points = []
try:
    for poly in intersecting_polygons:
        if poly.intersects(avoidance_directions):
            folium.features.GeoJson(data=poly.__geo_interface__,
                                    name='Busy Areas',
                                    style_function=style_function('#000c66'),
                                    overlay=True).add_to(map)

            if attempts < 10:
                avoided_points.append(poly)
                print("Number Unmerged Polygons: " + str(len(all_polygons)))
                print("Total Merged Polygons: " + str(len(intersecting_polygons)))
                print("Number Merged Polygons to API: " + str(len(avoided_points)))
                avoidance_route = create_route(coordinates, avoided_points, 1)
                avoidance_directions = create_buffer(avoidance_route)
                attempts += 1

                print('Generated alternative route, which avoids affected areas.')
            else:
                print('Too many reroutes')
                break
except Exception as e:
    print(e)
    
folium.features.GeoJson(
                data=avoidance_route,
                name='Alternative Route',
                style_function=style_function('#006600'),
                overlay=True).add_to(map)

folium.features.GeoJson(data=avoidance_directions,
                        name='Avoidance Directions Buffer',
                        style_function=style_function('#660061'),
                        overlay=True).add_to(map)

map.add_child(folium.map.LayerControl())
map

## Unmerged Routing

In [ ]:
# Create map
map = folium.Map(tiles='Stamen Toner', location=[40.76657321777155, -73.9831392189498], zoom_start=13)

#coordinates = [[-73.981386, 40.767868], [-73.973350, 40.764504]]

#coordinates = [[-73.992597, 40.763375], [-73.987212, 40.761124]]

#coordinates = [[-73.948918, 40.812560], [-73.944112, 40.810497]]

#coordinates = [[-73.94086360931398, 40.81892462451646], [-73.93900752067567, 40.82151464975118]]

#coordinates =[[-73.9422905445099, 40.820402331554625], [-73.93641114234926, 40.817893457797176]]

coordinates = [[-73.99379929637948, 40.7283915], [-73.9942, 40.7483]]

for coord in coordinates:
    folium.map.Marker(list(reversed(coord))).add_to(map)

# Initializations
now = datetime.now()
prediction_hour = "22"
prediction_day = 3

high_index_value_ls = []
point_geometry = []

# Get predicted locations
all_locations = predicted_locations(prediction_hour, prediction_day)

for location in all_locations:
    position = [location['long'], location['lat']]
    point_buffer = create_buffer_polygon(position)
    high_index_value_ls.append(point_buffer)
    point_geometry.append(Polygon(point_buffer))
    #folium.map.Marker(list(reversed(position)), 
    #                  icon=folium.Icon(color='lightgray', icon_color='blue',)
    #                  ,popup='High Count Value').add_to(map)

union_poly = unary_union(point_geometry)
union_poly_geojson = mapping(union_poly)

# Convert to GeoJSON-compatible format
union_poly_json = json.dumps(union_poly_geojson)
union_poly_data = json.loads(union_poly_json)

folium.features.GeoJson(data=union_poly_data,
                        name='Busy Areas',
                        style_function=style_function('#ffd699')).add_to(map)

avoided_point_list = []

# Create regular route with still empty avoided_point_list
optimal_directions = create_route(coordinates, avoided_point_list)

# Create buffer around route
avoidance_directions = create_buffer(optimal_directions)

folium.features.GeoJson(
                    data=optimal_directions,
                    name='Regular Route',
                    style_function=style_function('#ff5050'), overlay=True).add_to(map)

print('Generated regular route.')

# Initialise avoidance_route as an empty string
avoidance_route = ""
status = "no_rerouting"  # Indicates no rerouting was needed
attempts = 0
try:
    for site_poly in high_index_value_ls:
        poly = Polygon(site_poly)
        if poly.intersection(avoidance_directions):
            folium.features.GeoJson(data=poly,
                                        name='Busy Areas',
                                        style_function=style_function('#000c66'),
                                        overlay=True).add_to(map)
            # limits rerouting to 5 tries to preserve API quota and reduce processing time
            if attempts < 2:
                avoided_point_list.append(poly)
                avoidance_route = create_route(coordinates, avoided_point_list, 1)
                avoidance_directions = create_buffer(avoidance_route)
                attempts += 1
                # indicates routing was completed successfully, in under 5 attempts
                status = "rerouting_success"
                print('Generated alternative route, which avoids affected areas.')
            else:
                    # Indicates that too many attempts were made, and the avoidance route only avoids a maximum of 5 busy areas
                    break
except Exception as e:
    print(e)

if avoidance_route != "":    
    folium.features.GeoJson(
                    data=avoidance_route,
                    name='Alternative Route',
                    style_function=style_function('#006600'),
                    overlay=True).add_to(map)

    folium.features.GeoJson(data=avoidance_directions,
                            name='Avoidance Directions Buffer',
                            style_function=style_function('#660061'),
                            overlay=True).add_to(map)

map.add_child(folium.map.LayerControl())
map